<a href="https://colab.research.google.com/github/MoreiraErick/learning-python/blob/modulo-1-fundamentos-em-python/Bootcamp-IGTI-Aula-Interativa-1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Bootcamp Online - Desenvolvedor(a) Python IGTI**
# Aula Interativa 1
**Data:** 09/03/2021
**Professor:** Erick Faria
**Fonte:** Códigos adaptados de [codigoquant](https://gtihub.com/codigoquant)

# **Exemplo de aplicação do Python para Análise Financeira**

## Exemplo 1 - Criando um Ranking de Ações

## Exemplo 2 - Representação Gráfica da Carteira de Ações

## Exemplo 3 - Simulando uma carteira de Ações em comparação ao Ibovespa